# Landing Layer to Raw Layer (renaming columns, data type conversions, etc.)

In [3]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC") # fix timestamps loaded by spark
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/01 19:51:27 WARN Utils: Your hostname, DESKTOP-SATV84A resolves to a loopback address: 127.0.1.1; using 172.26.254.29 instead (on interface eth0)
23/08/01 19:51:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/01 19:51:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/01 19:51:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Yellow Taxis

In [30]:
# make directories for raw layer

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/raw/'):
    os.makedirs('../data/raw/')
    
if not os.path.exists('../data/raw/tlc'):
    os.makedirs('../data/raw/tlc')
    
if not os.path.exists('../data/raw/tlc/yellow'):
    os.makedirs('../data/raw/tlc/yellow')
    
if not os.path.exists('../data/raw/tlc/yellow/2021'):
    os.makedirs('../data/raw/tlc/yellow/2021')   
    
if not os.path.exists('../data/raw/tlc/yellow/2022'):
    os.makedirs('../data/raw/tlc/yellow/2022')
    
if not os.path.exists('../data/raw/tlc/yellow/2023'):
    os.makedirs('../data/raw/tlc/yellow/2023')

### Check data types

In [31]:
yellow_landing_dir = '../data/landing/tlc/yellow/'

sdf_yellow_2022_01 = spark.read.parquet(f"{yellow_landing_dir}2022/2022-01.parquet")

sdf_yellow_2022_all = spark.read.parquet(f"{yellow_landing_dir}2022")

sdf_yellow_2023_01 = spark.read.parquet(f"{yellow_landing_dir}2023/2023-01.parquet")

sdf_yellow_2023_02 = spark.read.parquet(f"{yellow_landing_dir}2023/2023-02.parquet")

#sdf_yellow_2022_all = spark.read.parquet('../data/raw/yellow/2022')



In [16]:
sdf_yellow_2022_01.schema

StructType([StructField('VendorID', LongType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', DoubleType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True)])

In [42]:
sdf_yellow_2022_all.schema

StructType([StructField('VendorID', LongType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', DoubleType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True)])

In [ ]:
sdf_yellow_2023_01.schema

In [ ]:
sdf_yellow_2023_02.schema

### Schemas appear to be incorrect for all of 2022, so we will adjust all datasets to line up with the schema of Feb 2023 data and ensure consistent lowercase casing

In [32]:
'''
# renaming
sdf.withColumnRenamed(
    'column_from',
    'column_to'
)

# example 1 for converting data types
sdf.withColumn(
    'column_to',
    F.col('column_from').cast('data type')
)
'''

# RatecodeID and passenger_count also need to be converted to int

sdf_yellow_2023_02 = sdf_yellow_2023_02.withColumn(
    'ratecodeid', 
    F.col('RatecodeID').cast('Integer') 
)

sdf_yellow_2023_02 = sdf_yellow_2023_02.withColumn(
    'passenger_count',
    F.col('passenger_count').cast('Integer')
)


sdf_yellow_2023_02.schema



StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('ratecodeid', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [33]:
# now, we want to ensure everything has consistent casing to make our lives easier
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in sdf_yellow_2023_02.columns]
sdf_yellow_2023_02 = sdf_yellow_2023_02.select(*consistent_col_casing)

# this will be used in the cell below when reading in
sdf_schema = sdf_yellow_2023_02.schema
sdf_schema

StructType([StructField('vendorid', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('ratecodeid', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('pulocationid', IntegerType(), True), StructField('dolocationid', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True)])

In [34]:
# convert all landing datasets to required schema

yellow_landing_dir = '../data/landing/tlc/yellow/'
    
yellow_raw_dir = '../data/raw/tlc/yellow/'

YEARS = ['2021', '2022', '2023']

for year in YEARS:
    if (year == '2021'):
        MONTHS = [12]
    if (year == '2022'):
        MONTHS = range(1,13)
    if (year == '2023'):   
        MONTHS = range(1,5) 
    for month in MONTHS:

        month = str(month).zfill(2) 

        sdf_malformed = spark.read.parquet(f"{yellow_landing_dir}{year}/{year}-{month}.parquet")

        # select all columns from the existing malformed dataframe and cast it to the required schema and change casing, drop ehail_fee as it has only NULL values
        sdf_malformed = sdf_malformed \
            .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)]) \
            .select(*consistent_col_casing) \
            .drop('ehail_fee')
        
        sdf_malformed \
        .coalesce(1) \
        .write \
        .mode('overwrite') \
        .parquet(f"{yellow_raw_dir}{year}/{year}-{month}.parquet")



In [35]:
# double check schema of 2022 raw layer data

sdf1 = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')

sdf1.show(1, vertical=True, truncate=100)

#sdf1.show(1, vertical=True, truncate=100)

''' checkinglanding vs raw dataset shape and landing dataset shape of jan 2022
sdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')

sdf2.show(1, vertical=True, truncate=100)

print(sdf1.count())

print(sdf2.count())

sdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')

sdf.show(1, vertical=True, truncate=100)

print(sdf.count())
'''

-RECORD 0------------------------------------
 vendorid              | 1                   
 tpep_pickup_datetime  | 2022-01-01 00:35:40 
 tpep_dropoff_datetime | 2022-01-01 00:53:29 
 passenger_count       | 2                   
 trip_distance         | 3.8                 
 ratecodeid            | 1                   
 store_and_fwd_flag    | N                   
 pulocationid          | 142                 
 dolocationid          | 236                 
 payment_type          | 1                   
 fare_amount           | 14.5                
 extra                 | 3.0                 
 mta_tax               | 0.5                 
 tip_amount            | 3.65                
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 21.95               
 congestion_surcharge  | 2.5                 
 airport_fee           | 0.0                 
only showing top 1 row



" checking raw vs cleaned dataset shape and raw dataset shape of jan 2022\nsdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')\n\nsdf2.show(1, vertical=True, truncate=100)\n\nprint(sdf1.count())\n\nprint(sdf2.count())\n\nsdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')\n\nsdf.show(1, vertical=True, truncate=100)\n\nprint(sdf.count())\n"

In [ ]:
# double check schema of 2023 raw layer data

sdf = spark.read.parquet('../data/cleaned/tlc/yellow/2023/*')

sdf.show(1, vertical=True, truncate=100)

## Green Taxis

In [36]:
# make directories for raw layer data

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/raw/'):
    os.makedirs('../data/raw/')
    
if not os.path.exists('../data/raw/tlc'):
    os.makedirs('../data/raw/tlc')
    
if not os.path.exists('../data/raw/tlc/green'):
    os.makedirs('../data/raw/tlc/green')
    
if not os.path.exists('../data/raw/tlc/green/2022'):
    os.makedirs('../data/raw/tlc/green/2022')
    
if not os.path.exists('../data/raw/tlc/green/2023'):
    os.makedirs('../data/raw/tlc/green/2023')

### Check data types

In [37]:
green_landing_dir = '../data/landing/tlc/green/'

sdf_green_2022_01 = spark.read.parquet(f"{green_landing_dir}2022/2022-01.parquet")

sdf_green_2022_all = spark.read.parquet(f"{green_landing_dir}2022")

sdf_green_2023_01 = spark.read.parquet(f"{green_landing_dir}2023/2023-01.parquet")

sdf_green_2023_02 = spark.read.parquet(f"{green_landing_dir}2023/2023-02.parquet")

#sdf_green_2022_all = spark.read.parquet('../data/raw/green/2022')



In [9]:
sdf_green_2022_01.schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', DoubleType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', IntegerType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', DoubleType(), True), StructField('trip_type', DoubleType(), True), StructField('congestion_surcharge', Doubl

In [47]:
sdf_green_2022_all.schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', DoubleType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', IntegerType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', DoubleType(), True), StructField('trip_type', DoubleType(), True), StructField('congestion_surcharge', Doubl

In [10]:
sdf_green_2023_01.schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', DoubleType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', IntegerType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', DoubleType(), True), StructField('trip_type', DoubleType(), True), StructField('congestion_surcharge', Doubl

In [49]:
sdf_green_2023_02.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', LongType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge', Doubl

In [38]:
sdf_green_2023_02.select('ehail_fee').distinct().collect()

sdf_green_2023_02.show(20)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       1| 2023-02-01 00:46:22|  2023-02-01 01:05:57|                 N|         1|          74|         265|              1|         10.8|       42.9|  1.0|    1.

### Schemas appear to be incorrect for all of 2022, so we will adjust all datasets to line up with the schema of Feb 2023 data and ensure consistent lowercase casing

In [39]:
'''
# renaming
sdf.withColumnRenamed(
    'column_from',
    'column_to'
)

# example 1 for converting data types
sdf.withColumn(
    'column_to',
    F.col('column_from').cast('data type')
)
'''

# RatecodeID and passenger_count also need to be converted to int

sdf_green_2023_02 = sdf_green_2023_02.withColumn(
    'ratecodeid', 
    F.col('RatecodeID').cast('Integer') 
)

sdf_green_2023_02 = sdf_green_2023_02.withColumn(
    'passenger_count',
    F.col('passenger_count').cast('Integer')
)


sdf_green_2023_02.schema



StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('ratecodeid', IntegerType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge',

In [40]:
# now, we want to ensure everything has consistent casing to make our lives easier
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in sdf_green_2023_02.columns]
sdf_green_2023_02 = sdf_green_2023_02.select(*consistent_col_casing)

# this will be used in the cell below when reading in
sdf_schema = sdf_green_2023_02.schema
sdf_schema

StructType([StructField('vendorid', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('ratecodeid', IntegerType(), True), StructField('pulocationid', IntegerType(), True), StructField('dolocationid', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge',

In [41]:
# convert all landing datasets to required schema

green_landing_dir = '../data/landing/tlc/green/'
    
green_raw_dir = '../data/raw/tlc/green/'

YEARS = ['2021', '2022', '2023']

for year in YEARS:
    if (year == '2021'):
        MONTHS = [12]
    if (year == '2022'):
        MONTHS = range(1,13)
    if (year == '2023'):   
        MONTHS = range(1,5) 
    for month in MONTHS:

        month = str(month).zfill(2) 

        sdf_malformed = spark.read.parquet(f"{green_landing_dir}{year}/{year}-{month}.parquet")

        # select all columns from the existing malformed dataframe and cast it to the required schema and change casing, drop ehail_fee as it has only NULL values
        sdf_malformed = sdf_malformed \
            .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)]) \
            .select(*consistent_col_casing) \
            .drop('ehail_fee')
        
        sdf_malformed \
        .coalesce(1) \
        .write \
        .mode('overwrite') \
        .parquet(f"{green_raw_dir}{year}/{year}-{month}.parquet")



In [42]:
# double check schema of 2022 raw layer data

sdf1 = spark.read.parquet('../data/raw/tlc/green/2022/*')

sdf1.show(1, vertical=True, truncate=100)

''' checking landing vs raw dataset shape and landing dataset shape of jan 2022
sdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')

sdf2.show(1, vertical=True, truncate=100)

print(sdf1.count())

print(sdf2.count())

sdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')

sdf.show(1, vertical=True, truncate=100)

print(sdf.count())
'''

-RECORD 0------------------------------------
 vendorid              | 1                   
 lpep_pickup_datetime  | 2022-03-01 00:24:14 
 lpep_dropoff_datetime | 2022-03-01 00:34:04 
 store_and_fwd_flag    | N                   
 ratecodeid            | 1                   
 pulocationid          | 74                  
 dolocationid          | 151                 
 passenger_count       | 1                   
 trip_distance         | 2.3                 
 fare_amount           | 9.5                 
 extra                 | 0.5                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 10.8                
 payment_type          | 2                   
 trip_type             | 1                   
 congestion_surcharge  | 0.0                 
only showing top 1 row



" checking raw vs cleaned dataset shape and raw dataset shape of jan 2022\nsdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')\n\nsdf2.show(1, vertical=True, truncate=100)\n\nprint(sdf1.count())\n\nprint(sdf2.count())\n\nsdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')\n\nsdf.show(1, vertical=True, truncate=100)\n\nprint(sdf.count())\n"

In [29]:
# double check schema of 2023 raw layer data

sdf = spark.read.parquet('../data/raw/tlc/green/2023/*')

sdf.show(1, vertical=True, truncate=100)

-RECORD 0------------------------------------
 vendorid              | 2                   
 lpep_pickup_datetime  | 2023-03-01 00:25:10 
 lpep_dropoff_datetime | 2023-03-01 00:35:47 
 store_and_fwd_flag    | N                   
 ratecodeid            | 1                   
 pulocationid          | 82                  
 dolocationid          | 196                 
 passenger_count       | 1                   
 trip_distance         | 2.36                
 fare_amount           | 13.5                
 extra                 | 1.0                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 improvement_surcharge | 1.0                 
 total_amount          | 16.0                
 payment_type          | 2                   
 trip_type             | 1                   
 congestion_surcharge  | 0.0                 
only showing top 1 row



## Taxi Zones

In [75]:
# make directories for raw layer

import os

if not os.path.exists('../data/raw/'):
    os.makedirs('../data/raw/')
    
if not os.path.exists('../data/raw/tlc'):
    os.makedirs('../data/raw/tlc')
    
if not os.path.exists('../data/raw/tlc/taxi_zones'):
    os.makedirs('../data/raw/tlc/taxi_zones')

In [5]:
taxi_zone_lookup = spark.read.option("header",True) \
                   .csv("../data/landing/tlc/taxi_zones/taxi+_zone_lookup.csv")

In [10]:
print(taxi_zone_lookup.limit(5))

print(taxi_zone_lookup.schema)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+

StructType([StructField('LocationID', StringType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])


### Just need to convert the column names to lowercase for this data

In [12]:
from pyspark.sql import functions as F

# converting to lower case
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in taxi_zone_lookup.columns]
taxi_zone_lookup = taxi_zone_lookup.select(*consistent_col_casing)

print(taxi_zone_lookup.limit(5))
print(taxi_zone_lookup.schema)

+----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+

StructType([StructField('locationid', StringType(), True), StructField('borough', StringType(), True), StructField('zone', StringType(), True), StructField('service_zone', StringType(), True)])


In [13]:
# save raw layer data as parquet
taxi_zone_lookup \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet('../data/raw/tlc/taxi_zones/taxi_zone_lookup.parquet')

In [14]:
# check raw layer data
taxi_zone_lookup = spark.read.parquet('../data/raw/tlc/taxi_zones/taxi_zone_lookup.parquet')

taxi_zone_lookup.printSchema


<bound method DataFrame.printSchema of +----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City

In [27]:
# move taxi_zones.zip to raw layer data
import shutil

src = '../data/landing/tlc/taxi_zones/taxi_zones.zip'

dst = '../data/raw/tlc/taxi_zones/taxi_zones.zip'

shutil.copyfile(src, dst)

import zipfile

# unzip shapefile
with zipfile.ZipFile(dst, 'r') as zip_ref:
    zip_ref.extractall('../data/raw/tlc/taxi_zones')

## External Datasets

### Motor Vehicle Collisions

### Subway Entrances and Exits

### Subway Hourly Ridership

### Bus Hourly Ridership

### Hotel Properties

### Airbnb

### Census